# Visualize rejuvenation in the core using $\zeta$ Ophiuchi's models
author: [Mathieu Renzo](mailto:mrenzo@flatironinstitute.org)

We will compare the mass-fraction profile of $^1\mathrm{H}$ in an
accretor star going from $17\rightarrow 20\,M_\o dot$during Roche lobe
overflow. The models are from [Renzo & Gotbergo
2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...923..277R/abstract).

We chose a single star of the same total post-RLOF mass as in [Renzo et al. 2022](https://ui.adsabs.harvard.edu/abs/2022arXiv220615338R/abstract)

In [ ]:
import sys
# the line below import stuff from git@github.com:mathren90/plotFunc.git
sys.path.append('/mnt/home/mrenzo/codes/python_stuff/plotFunc/')
from MESAreader import *
import matplotlib.pyplot as plt
%matplotlib inline
from plot_defaults import *
# plot_aux defines many auxiliary functions to make plots
from plot_aux import *


In [ ]:
set_plot_defaults_from_matplotlibrc('./')

In [ ]:
bin_folder = "/mnt/home/mrenzo/ceph/RESULTS/zeta_oph/zeta_ophiuchi/M1_25_M2_17_P100_Z0.01/re-run_exponential/"
acc_folder = bin_folder+'/binary_to_postRLOF/LOGS2/'
acc_post_RLOF = bin_folder+'/accretor_to_TAMS/LOGS/'
single_folder = "/mnt/home/mrenzo/ceph/RESULTS/zeta_oph/zeta_ophiuchi/single_star/Z0.01/20/non_rot/LOGS/"

In [ ]:
accretor_pfiles = sorted(glob.glob(acc_folder+"profile*.data"), key=get_age_from_profile)+\
    sorted(glob.glob(acc_post_RLOF+'profile*.data'), key=get_age_from_profile)
print(len(accretor_pfiles))

single_pfiles = sorted(glob.glob(single_folder+"/profile*.data"), key=get_age_from_profile)
print(len(single_pfiles))

In [ ]:
# get RLOF phase
bsrc, bcol = getSrcCol(acc_folder+"/../binary_history.data")
iRLOF = (bsrc[:, bcol.index("rl_relative_overflow_1")] >0)
tRLOF = bsrc[iRLOF, bcol.index("age")]
tRLOF_start = min(tRLOF)
tRLOF_end = max(tRLOF)
src_2, col_2 = getSrcCol(acc_post_RLOF+'/history.data')
tTAMS_acc = max(src_2[:, col_2.index('star_age')])
print(tRLOF_start/1e6, tRLOF_end/1e6)

In [ ]:
cmap1, vmin1, vmax1, norm1, cmap2, vmin2, vmax2, norm2, cmap3, vmin3, vmax3, norm3 = get_three_colormaps(tRLOF_start,
                                                                                                         tRLOF_end,
                                                                                                         tTAMS_acc)

In [ ]:
def plot_H_mass(pfile, ax, **plot_kwargs):
    """ plot mass fraction profile from profile*.data"""
    src, col = getSrcCol(pfile)
    h1 = src[:, col.index("h1")]
    m = src[:, col.index("mass")]
    line = ax.plot(m, h1, **plot_kwargs)
    return line    

In [ ]:
fig = plt.figure(figsize=(10,10))
gs = gridspec.GridSpec(120, 100)
ax1 = fig.add_subplot(gs[:,:])
ax1.set_xlabel(r"$Mass \ [M_\odot]$")
ax1.set_ylabel(r"$\mathrm{Mass\ fraction}\ X(^1\mathrm{H})$")
ax1.set_ylim(0, 1)
ax1.set_xlim(0, 21.5)
ax1.text(4.2, 1.03, r"$\leftarrow$ center", transform=ax1.transData, fontsize=25, color="#808080", ha="right", va="center")
ax1.text(17, 1.03, r"surface $\rightarrow$", transform=ax1.transData, fontsize=25, color="#808080", ha="left", va="center")


# single stars
# legend
ax1.plot(np.nan, np.nan, c='k', ls='--', lw=2, label=r"single $20\,M_\odot$")
ax1.legend()

colors = plt.cm.viridis(np.linspace(0.2,1,6))
to_plot = single_pfiles[::20]+[single_pfiles[-1]]
for p in to_plot:
    i = to_plot.index(p)
    line = plot_H_mass(p, ax1, ls='--', lw=2, color=colors[i], zorder=10)
    age = get_age_from_profile(p)
    age_label = ax1.text(0.8, 0.2, f"{age:.5f}"+r"$\,\mathrm{Myr}$", fontsize=30, va="center", ha="center", transform=ax1.transAxes, color=colors[i])
    plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/"+f"{i:04d}"+"single_only.png")
    plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/"+f"{i:04d}"+"single_only.pdf")
    age_label.set_visible(False)
    
ax1.plot(np.nan, np.nan, c='orange', ls='-', lw=5, label=r"accretor $17\rightarrow 20\,M_\odot$")
ax1.legend()

# to_plot = [acc_folder+'/profile500.data',
#            acc_folder+'/profile3850.data',
#            acc_folder+'/profile6500.data',
#            acc_post_RLOF+'/profile496.data',
#            acc_post_RLOF+'/profile893.data']
to_plot = [accretor_pfiles[0]]+accretor_pfiles[::1000]+[accretor_pfiles[-1]]
for p in to_plot:
    i = to_plot.index(p)
    line = plot_H_mass(p, ax1, ls='-', lw=5, color="orange", zorder=1)
    age = get_age_from_profile(p)
    age_label = ax1.text(0.8, 0.2, f"{age:.5f}"+r"$\,\mathrm{Myr}$", fontsize=30, va="center", ha="center", transform=ax1.transAxes, color="orange")
    if (i !=0) and (i != len(to_plot)-1):
        plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/"+f"{i:04d}"+"acc.png")
        plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/"+f"{i:04d}"+"acc.pdf")
    elif i ==0:
        plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/0000acc_ZAMS.png")
        plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/0000acc_ZAMS.pdf")
    else:
        plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/0000acc_TAMS.png")
        plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/0000acc_TAMS.pdf")

    line.pop(0).remove()
    age_label.set_visible(False)


# development plots

In [ ]:
def plot_rho_mass(pfile, ax, **plot_kwargs):
    """ plot density profile from profile*.data"""
    src, col = getSrcCol(pfile)
    log_rho = src[:, col.index("logRho")]
    rho = 10.**log_rho
    m = src[:, col.index("mass")]
    line = ax.plot(m, rho, **plot_kwargs)
    return line


In [ ]:
fig = plt.figure(figsize=(15,10))
gs = gridspec.GridSpec(100, 100)
ax2 = fig.add_subplot(gs[:50,:])
ax1 = fig.add_subplot(gs[50:,:])
ax1.set_xlabel(r"$Mass \ [M_\odot]$")
ax1.set_ylabel(r"$X(^1\mathrm{H})$")
ax2.set_ylabel(r"$\rho \ [\mathrm{g\ cm^{-3}}]$")
ax1.set_ylim(0, 1)
ax1.set_xlim(0, 21.5)
ax2.set_ylim(ymin=1e-12, ymax=1e3)
ax2.set_yscale('log')
ax2.set_xlim(0, 21.5)
ax2.text(4.2, 2.05, r"$\leftarrow$ center", transform=ax1.transData, fontsize=25, color="#808080", ha="right", va="center")
ax2.text(17, 2.05, r"surface $\rightarrow$", transform=ax1.transData, fontsize=25, color="#808080", ha="left", va="center")
ax2.set_xticklabels([])
ax1.set_yticks([0, 0.2, 0.4, 0.6, 0.8])
ax1.set_yticks([0.1, 0.3, 0.5, 0.7, 0.9], minor=True)
# inset plot
zx = inset_axes(ax2, 8, 2, bbox_to_anchor=(17,5e-2),
                bbox_transform=ax2.transData)




# single stars
# legend
ax1.plot(np.nan, np.nan, c='k', ls='--', lw=2, label=r"single $20\,M_\odot$")
ax1.legend(ncol=2,loc="upper right")

colors = plt.cm.viridis(np.linspace(0,1,6))
to_plot = single_pfiles[::20]+[single_pfiles[-1]]
for p in to_plot:
    i = to_plot.index(p)
    plot_H_mass(p, ax1, ls='--', lw=2, color=colors[i], zorder=10)
    plot_rho_mass(p, ax2, ls='--', lw=2, color=colors[i], zorder=10)
    plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/"+f"{i:04d}"+"rho_X_single_only.png")

ax1.plot(np.nan, np.nan, c='orange', ls='-', label=r"accretor $17\rightarrow 20\,M_\odot$")
ax1.legend(ncol=2,loc="upper right")

# to_plot = [acc_folder+'/profile500.data',
#            acc_folder+'/profile3850.data',
#            acc_folder+'/profile6500.data',
#            acc_post_RLOF+'/profile496.data',
#            acc_post_RLOF+'/profile893.data']
to_plot = accretor_pfiles[::1000]
for p in to_plot:
    i = to_plot.index(p)
    line = plot_H_mass(p, ax1, ls='-', color="orange", zorder=1)
    line_rho = plot_rho_mass(p, ax2, ls='--', lw=2, color="orange", zorder=10)
    line_rho_inset = plot_rho_mass(p, zx, ls='--', lw=2, color="orange", zorder=10)
    age = get_age_from_profile(p)
    age_label = ax1.text(0.8, 0.2, f"{age:.5f}"+r"$\,\mathrm{Myr}$", fontsize=30, va="center", ha="center", transform=ax1.transAxes, color="orange")
    plt.savefig("/mnt/home/mrenzo/TMP/rejuvenation_plots/"+f"{i:04d}"+"rho_X_acc.png")
    line.pop(0).remove()
    line_rho.pop(0).remove()
    line_rho_inset.pop(0).remove()
    age_label.set_visible(False)



In [ ]:
print(type(zx))